In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import UndefinedMetricWarning
import os
import time

import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Define device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory to save the best models
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Define training loop
def train(model, train_loader, val_loader, epochs=10, patience=5):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation
        val_loss = validate(model, val_loader, criterion)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')
        
        model_path = os.path.join(models_dir, f'best_model_{name.replace(" ", "_").lower()}.pth')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping!')
                model.load_state_dict(torch.load(model_path))
                return

def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1).float())
            total_loss += loss.item()
    return total_loss / len(val_loader)


def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs) > 0.5
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print('Accuracy:', accuracy)
    print('Classification Report:')
    print(classification_report(all_labels, all_preds))
    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, all_preds))
    return accuracy

# Load data
data_dir = 'sequences'
data = torch.load(os.path.join(data_dir, 'data_01.pt'))
X = data['X']
y = data['y']

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Create DataLoaders
batch_size = 16
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, x):
        raise NotImplementedError

class FlattenModel(BaseModel):
    def forward(self, x):
        return self._forward(x.view(x.size(0), -1))

    def _forward(self, x):
        raise NotImplementedError

class LogisticRegression(FlattenModel):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def _forward(self, x):
        return self.linear(x)

class MLP(FlattenModel):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def _forward(self, x):
        return self.layers(x)

class LSTMModel(BaseModel):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])

class BidirectionalLSTMModel(BaseModel):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BidirectionalLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)  # *2 because it's bidirectional
    
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        x = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        return self.fc(x)


# Update the models dictionary
input_dim = 64 * 8  # Assuming your input shape is (batch_size, 64, 8)
models = {
    'Logistic Regression': LogisticRegression(input_dim),
    'MLP': MLP(input_dim),
    'LSTM': LSTMModel(input_size=8, hidden_size=128, num_layers=1),
    'Bidirectional LSTM': BidirectionalLSTMModel(input_size=8, hidden_size=128, num_layers=1),
}


# Train and evaluate each model
results = {}
for name, model in models.items():
    print(f"\nTraining {name}")
    start_time = time.time()
    train(model, train_loader, val_loader, epochs=500, patience=5)
    train_time = time.time() - start_time
    
    print(f"\nEvaluating {name}")
    accuracy = evaluate(model, test_loader)
    
    results[name] = {'accuracy': accuracy, 'train_time': train_time}

# Print summary of results
print("\nSummary of Results:")
for name, result in results.items():
    print(f"{name}: Accuracy = {result['accuracy']:.4f}, Training Time = {result['train_time']:.2f} seconds")


Training Logistic Regression
Epoch 10, Train Loss: 0.5594, Val Loss: 0.4377
Epoch 20, Train Loss: 0.4973, Val Loss: 0.3828
Epoch 30, Train Loss: 0.4631, Val Loss: 0.3587
Epoch 40, Train Loss: 0.4298, Val Loss: 0.3007
Epoch 50, Train Loss: 0.4124, Val Loss: 0.2845
Epoch 60, Train Loss: 0.3988, Val Loss: 0.2830
Early stopping!

Evaluating Logistic Regression
Accuracy: 0.8793103448275862
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      1.00      0.93        44
         1.0       1.00      0.50      0.67        14

    accuracy                           0.88        58
   macro avg       0.93      0.75      0.80        58
weighted avg       0.90      0.88      0.86        58

Confusion Matrix:
[[44  0]
 [ 7  7]]

Training MLP
Epoch 10, Train Loss: 0.4280, Val Loss: 0.3077
Epoch 20, Train Loss: 0.3472, Val Loss: 0.2582
Epoch 30, Train Loss: 0.3545, Val Loss: 0.1974
Epoch 40, Train Loss: 0.2865, Val Loss: 0.1729
Early stopping!

Evalu